## Python `Protocol` 類別

在 Python 中，`Protocol` 類別（定義於 `typing` 模組中）是用來支援「結構型子類型檢查」（structural subtyping，也有人稱作靜態的「Duck Typing」）的一種工具。它讓你可以像定義傳統的介面（interface）一樣，宣告一組方法與屬性，只要某個物件「形狀吻合」── 也就是擁有這些方法與屬性── 就會被視為符合該 Protocol，而不必在繼承樹上有明確的繼承關係。

### 主要特點

- **結構型檢查**  
  不需要明確繼承，只要物件實現了 Protocol 中宣告的所有成員，就可認定為符合該介面。  
- **靜態類型檢查**  
  在使用 `mypy`、`Pyright` 等靜態檢查工具時，可以針對 Protocol 做型別檢查。  
- **可選的執行時檢查**  
  搭配 `@runtime_checkable`，可於執行時透過 `isinstance(obj, SomeProtocol)` 做簡易檢查（僅限方法／屬性存在性檢測）。

### 範例

In [1]:
from typing import Protocol, runtime_checkable


# 定義一個 Protocol，宣告需要有 'read' 與 'write' 方法
@runtime_checkable
class ReaderWriter(Protocol):
    def read(self, n: int) -> bytes: ...

    def write(self, data: bytes) -> int: ...


# 任意類別，只要實作了 read 與 write，就「形狀吻合」
class FileLike:
    def read(self, n: int) -> bytes:
        return b"data"

    def write(self, data: bytes) -> int:
        print(f"FileLike write(): {data}")
        return len(data)


def process(rw: ReaderWriter) -> None:
    chunk = rw.read(1024)
    rw.write(chunk)


f = FileLike()
process(f)  # 不用繼承，也能通過靜態檢查與運行


# 執行時檢查示範
@runtime_checkable
class SupportsClose(Protocol):
    def close(self) -> None: ...


class NoClose:
    pass


# 靜態檢查有效，但執行時為 False
print(f"isinstance(f, ReaderWriter): {isinstance(f, ReaderWriter)}")
# 執行時檢查
print(f"isinstance(f, SupportsClose): {isinstance(f, SupportsClose)}")  # False

FileLike write(): b'data'
isinstance(f, ReaderWriter): True
isinstance(f, SupportsClose): False


### 何時使用 Protocol？

- 需要定義「只要形狀吻合就可以使用」的介面（例如各種檔案-或網路-類似物件）。

- 希望靜態型別檢查工具能對不同類別進行統一的介面檢查，卻不想強制它們繼承同一個基底。

- 想要更細緻地表達函式參數或回傳值要求的「結構」而非「命名繼承」。

Protocol 是 PEP 544 所引入的特性，自 Python 3.8 起可使用，屬於進階型別提示工具箱中的重要一員。